In [2]:
# !/wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

In [3]:
# !unzip -q new_articles.zip -d new_articles

In [4]:
import os
os.environ['OPENAI_API_KEY']=''



In [5]:
from langchain.vectorstores import  Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [6]:
loader=DirectoryLoader('new_articles',glob='./*.txt',loader_cls=TextLoader)

In [7]:
document=loader.load()

In [8]:
document[0].page_content

'Welcome to The Interchange! If you received this in your inbox, thank you for signing up and your vote of confidence. If you’re reading this as a post on our site, sign up here so you can receive it directly in the future. Every week, we’ll take a look at the hottest fintech news of the previous week. This will include everything from funding rounds to trends to an analysis of a particular space to hot takes on a particular company or phenomenon. There’s a lot of fintech news out there and it’s our job to stay on top of it — and make sense of it — so you can stay in the know. — Mary Ann and Christine\n\nBusy, busy, busy\n\nIt was a busy week in startup and venture lands, and the fintech space was no exception.\n\nIn the venture world, I reported on Peter Ackerson’s departure from Fin Capital earlier this year and the fact that he has since started a new venture firm called Audere Capital. The circumstances around his departure remain fuzzy, but one source speculated that tension arose

In [9]:
from langchain.text_splitter import  RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
text=text_splitter.split_documents(document)
#chunk size means the no of characters and the chunk overalpap means  the prev chunk size we will take

In [10]:
text[0].page_content

'Welcome to The Interchange! If you received this in your inbox, thank you for signing up and your vote of confidence. If you’re reading this as a post on our site, sign up here so you can receive it directly in the future. Every week, we’ll take a look at the hottest fintech news of the previous week. This will include everything from funding rounds to trends to an analysis of a particular space to hot takes on a particular company or phenomenon. There’s a lot of fintech news out there and it’s our job to stay on top of it — and make sense of it — so you can stay in the know. — Mary Ann and Christine\n\nBusy, busy, busy\n\nIt was a busy week in startup and venture lands, and the fintech space was no exception.'

In [11]:
len(text)

233

In [12]:
#CREATING DB
persist_directory='db'

In [13]:
from langchain import embeddings
embeddings=OpenAIEmbeddings()

/home/seaflux/VectorDB/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [14]:
vectordb=Chroma.from_documents(documents=text,embedding=embeddings,persist_directory=persist_directory)

In [15]:
vectordb

In [16]:
vectordb.persist()

/home/seaflux/VectorDB/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [17]:
vectordb=None

In [18]:
vectordb=Chroma.from_documents(documents=text,embedding=embeddings,persist_directory=persist_directory)

In [19]:
#Make a retriver
retiriever=vectordb.as_retriever()

In [20]:
retiriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f9d2f128410>)

In [21]:
docs=retiriever.get_relevant_documents('what is ai')

/home/seaflux/VectorDB/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [22]:
docs

[Document(page_content='AI startup Hugging Face and ServiceNow Research, ServiceNow’s R&D division, have released StarCoder, a free alternative to code-generating AI systems along the lines of GitHub’s Copilot.\n\nCode-generating systems like DeepMind’s AlphaCode; Amazon’s CodeWhisperer; and OpenAI’s Codex, which powers Copilot, provide a tantalizing glimpse at what’s possible with AI within the realm of computer programming. Assuming the ethical, technical and legal issues are someday ironed out (and AI-powered coding tools don’t cause more bugs and security exploits than they solve), they could cut development costs substantially while allowing coders to focus on more creative tasks.', metadata={'source': 'new_articles/05-04-hugging-face-and-servicenow-release-a-free-code-generating-model.txt'}),
 Document(page_content='AI startup Hugging Face and ServiceNow Research, ServiceNow’s R&D division, have released StarCoder, a free alternative to code-generating AI systems along the lines 

In [23]:
#Make a retriver
retiriever2=vectordb.as_retriever(search_kwargs={'k':2})

In [24]:
retiriever2.get_relevant_documents('what is ai')

[Document(page_content='AI startup Hugging Face and ServiceNow Research, ServiceNow’s R&D division, have released StarCoder, a free alternative to code-generating AI systems along the lines of GitHub’s Copilot.\n\nCode-generating systems like DeepMind’s AlphaCode; Amazon’s CodeWhisperer; and OpenAI’s Codex, which powers Copilot, provide a tantalizing glimpse at what’s possible with AI within the realm of computer programming. Assuming the ethical, technical and legal issues are someday ironed out (and AI-powered coding tools don’t cause more bugs and security exploits than they solve), they could cut development costs substantially while allowing coders to focus on more creative tasks.', metadata={'source': 'new_articles/05-04-hugging-face-and-servicenow-release-a-free-code-generating-model.txt'}),
 Document(page_content='AI startup Hugging Face and ServiceNow Research, ServiceNow’s R&D division, have released StarCoder, a free alternative to code-generating AI systems along the lines 

In [25]:
#Make a chain
from langchain.chains import RetrievalQA
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retiriever,
                                  return_source_documents=True)

/home/seaflux/VectorDB/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [26]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [27]:
# full example
query = "How much money did Microsoft raise?"

In [28]:
llm_response = qa_chain(query)

/home/seaflux/VectorDB/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [29]:
llm_response

{'query': 'How much money did Microsoft raise?',
 'result': ' Microsoft did not raise any money in this context. Other companies and venture capital firms raised money, but there is no mention of Microsoft raising money in this context.',
 'source_documents': [Document(page_content='April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes the forthcoming offering as “for professionals who need more c

In [30]:
process_llm_response(llm_response)

 Microsoft did not raise any money in this context. Other companies and venture capital firms raised money, but there is no mention of Microsoft raising money in this context.


Sources:
new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt
new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt
new_articles/05-07-3one4-capital-driven-by-contrarian-bets-raises-200-million-new-fund.txt
new_articles/05-07-3one4-capital-driven-by-contrarian-bets-raises-200-million-new-fund.txt


In [31]:
!zip -r db.zip ./db

updating: db/ (stored 0%)
updating: db/chroma-embeddings.parquet (deflated 29%)
updating: db/index/ (stored 0%)
updating: db/chroma-collections.parquet (deflated 50%)
  adding: db/index/id_to_uuid_5ee6ba7b-d7c7-46b4-bdd3-4cf417bd1fd7.pkl (deflated 35%)
  adding: db/index/index_metadata_5ee6ba7b-d7c7-46b4-bdd3-4cf417bd1fd7.pkl (deflated 14%)
  adding: db/index/index_5ee6ba7b-d7c7-46b4-bdd3-4cf417bd1fd7.bin (deflated 17%)
  adding: db/index/uuid_to_id_5ee6ba7b-d7c7-46b4-bdd3-4cf417bd1fd7.pkl (deflated 39%)


In [32]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

In [33]:

# delete the directory
!rm -rf db/